# Get Data from NBA Official API

In [1]:
from nba_api.stats.static import players


In [2]:
import os
import pandas as pd  # Force load pandas
from nba_api.stats.endpoints import shotchartdetail
import json
import requests

# Continue with your code


Pandas imported successfully in nba_api, version: 2.2.2


In [3]:
active_players = players.get_active_players()

In [4]:
len(active_players)

531

In [6]:
# Storing Directory for All Players
player_dictionary = players.get_active_players()
# Returning first 5 players in player_dictionary
player_dictionary[0:5]

[{'id': 1630173,
  'full_name': 'Precious Achiuwa',
  'first_name': 'Precious',
  'last_name': 'Achiuwa',
  'is_active': True},
 {'id': 203500,
  'full_name': 'Steven Adams',
  'first_name': 'Steven',
  'last_name': 'Adams',
  'is_active': True},
 {'id': 1628389,
  'full_name': 'Bam Adebayo',
  'first_name': 'Bam',
  'last_name': 'Adebayo',
  'is_active': True},
 {'id': 1630534,
  'full_name': 'Ochai Agbaji',
  'first_name': 'Ochai',
  'last_name': 'Agbaji',
  'is_active': True},
 {'id': 1630583,
  'full_name': 'Santi Aldama',
  'first_name': 'Santi',
  'last_name': 'Aldama',
  'is_active': True}]

In [6]:
from nba_api.stats.static import players
import json

# Get all players.
all_players = players.get_players()

# Filter active players.
active_players = [player for player in all_players if player['is_active']]

# Save the active players to a JSON file.
with open('updated_active_players.json', 'w') as f:
    json.dump(active_players, f, indent=4)

print("Updated active player data has been saved.")


Updated active player data has been saved.


In [1]:
from nba_api.stats.static import players
import pandas as pd

# Get all players.
all_players = players.get_players()

# Filter active players.
active_players = [player for player in all_players if player['is_active']]

# Convert the list of active players to a DataFrame
active_players_df = pd.DataFrame(active_players)

# Save the active players to a CSV file.
active_players_df.to_csv('updated_active_players.csv', index=False)

print("Updated active player data has been saved as a CSV file.")


Updated active player data has been saved as a CSV file.


In [9]:

# # Define the seasons you want to fetch data for
# seasons = ['2022-23', '2023-24']


# # Iterate through the selected players and seasons
# for select_player in active_players[79:]:
#     player_all_seasons_shots = pd.DataFrame()  # DataFrame to store a player's shots across seasons
    
#     for season in seasons:
#         i = 0
#         while i < 3:
#             try:
#                 allshots = shotchartdetail.ShotChartDetail(
#                     team_id=0, 
#                     player_id=select_player, 
#                     context_measure_simple='FGA',
#                     season_nullable=season
#                 )
#                 player_season_shots = allshots.get_data_frames()[0]
#                 player_all_seasons_shots = pd.concat([player_all_seasons_shots, player_season_shots])
#                 break
#             except Exception as e:
#                 i += 1
        
#         if i == 3:
#             print(f"Connection Error after trying 3 times for ({select_player}) in season {season}")
#             # Decide if you want to continue with the next season or break
#             continue  # Proceed with the next season
    
#     # After collecting shots from all seasons, check if data exists and save to a CSV file
#     if player_all_seasons_shots.shape[0] > 200:
#         player_all_seasons_shots.to_csv(f'shot_data/{select_player}.csv', index=False)


In [9]:
from nba_api.stats.endpoints import shotchartdetail, commonteamroster
import pandas as pd
import time


# Define the seasons you want to fetch data for
seasons = ['2022-23', '2023-24']
miami_heat_team_id = 1610612748

def get_miami_heat_roster(team_id, season):
    roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
    return roster.get_data_frames()[0]


roster = get_miami_heat_roster(miami_heat_team_id, seasons)
roster

for select_player in active_players[:]:
    player_id = select_player['id']  # Extract player ID correctly
    player_all_seasons_shots = pd.DataFrame()
    season_data_retrieved = set()  # Set to keep track of seasons with successful data retrieval

    for season in seasons:
        attempt = 0
        while attempt < 3:
            try:
                allshots = shotchartdetail.ShotChartDetail(
                    team_id=miami_heat_team_id,
                    player_id=player_id,
                    context_measure_simple='FGA',
                    season_type_all_star = ['Regular Season', 'Playoffs'],
                    season_nullable=season
                )
                player_season_shots = allshots.get_data_frames()[0]
                if not player_season_shots.empty:
                    player_all_seasons_shots = pd.concat([player_all_seasons_shots, player_season_shots])
                    season_data_retrieved.add(season)  # Add season to the set if data was retrieved
                break
            except Exception as e:
                print(f"Attempt {attempt+1} failed for player {player_id} in season {season}: {str(e)}")
                attempt += 1

        if attempt == 3:
            print(f"Failed to connect after 3 attempts for player {player_id} in season {season}")

    # Check if data for both seasons was successfully retrieved before saving
    if len(season_data_retrieved) == len(seasons) and player_all_seasons_shots.shape[0] >= 50:
        player_all_seasons_shots.to_csv(f'shot_data/{player_id}.csv', index=False)
        print(f"Data for player {player_id} from seasons {', '.join(seasons)} saved successfully.")
    else:
        print(f"Insufficient data for player {player_id}; data not saved.")


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonteamroster?LeagueID=&Season=2022-23&Season=2023-24&TeamID=1610612748 HTTP/1.1" 200 1526
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID', 'HOW_ACQUIRED']
Headers (simple): ['TEAM_ID', 'SEASON', 'COACH_ID', 'FIRST_NAME', 'LAST_NAME', 'COACH_NAME', 'IS_ASSISTANT', 'COACH_TYPE', 'SORT_SEQUENCE', 'SUB_SORT_SEQUENCE']


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/shotchartdetail?AheadBehind=&ClutchTime=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&EndPeriod=&EndRange=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&Period=0&PlayerID=1630173&PlayerPosition=&PointDiff=&Position=&RangeType=&RookieYear=&Season=2022-23&SeasonSegment=&SeasonType=Regular+Season&SeasonType=Playoffs&StartPeriod=&StartRange=&TeamID=1610612748&VsConference=&VsDivision= HTTP/1.1" 200 998
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM']
Headers (simple): ['GRID_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'FGA', 'FGM', 'FG_PCT']


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/shotchartdetail?AheadBehind=&ClutchTime=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&EndPeriod=&EndRange=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&Period=0&PlayerID=1630173&PlayerPosition=&PointDiff=&Position=&RangeType=&RookieYear=&Season=2023-24&SeasonSegment=&SeasonType=Regular+Season&SeasonType=Playoffs&StartPeriod=&StartRange=&TeamID=1610612748&VsConference=&VsDivision= HTTP/1.1" 200 1011
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


...

Headers (simple): ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM']
Headers (simple): ['GRID_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'FGA', 'FGM', 'FG_PCT']
Insufficient data for player 203469; data not saved.


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/shotchartdetail?AheadBehind=&ClutchTime=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&EndPeriod=&EndRange=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&Period=0&PlayerID=1627826&PlayerPosition=&PointDiff=&Position=&RangeType=&RookieYear=&Season=2022-23&SeasonSegment=&SeasonType=Regular+Season&SeasonType=Playoffs&StartPeriod=&StartRange=&TeamID=1610612748&VsConference=&VsDivision= HTTP/1.1" 200 997
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM']
Headers (simple): ['GRID_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'FGA', 'FGM', 'FG_PCT']


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/shotchartdetail?AheadBehind=&ClutchTime=&ContextFilter=&ContextMeasure=FGA&DateFrom=&DateTo=&EndPeriod=&EndRange=&GameID=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&Period=0&PlayerID=1627826&PlayerPosition=&PointDiff=&Position=&RangeType=&RookieYear=&Season=2023-24&SeasonSegment=&SeasonType=Regular+Season&SeasonType=Playoffs&StartPeriod=&StartRange=&TeamID=1610612748&VsConference=&VsDivision= HTTP/1.1" 200 1010


Headers (simple): ['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME', 'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING', 'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE', 'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE', 'HTM', 'VTM']
Headers (simple): ['GRID_TYPE', 'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'FGA', 'FGM', 'FG_PCT']
Insufficient data for player 1627826; data not saved.


In [2]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
import pandas as pd
from tqdm import tqdm

# Fetch all NBA teams
all_teams = teams.get_teams()

# Define the seasons you are interested in
seasons = ['2022-23', '2023-24']

# Function to fetch rosters for all teams and all specified seasons
def get_all_teams_rosters(all_teams, seasons):
    # DataFrame to store all rosters
    all_rosters = pd.DataFrame()
    
    # Loop over each team
    for team in tqdm(all_teams, desc="Processing teams"):
        team_id = team['id']
        team_name = team['full_name']
        
        # Loop over each season
        for season in tqdm(seasons, desc=f"Fetching rosters for {team_name}", leave=False):
            try:
                roster_response = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
                roster_data = roster_response.get_data_frames()[0]
                roster_data['Season'] = season  # Add season info
                roster_data['TeamName'] = team_name  # Add team name info
                all_rosters = pd.concat([all_rosters, roster_data], ignore_index=True)
            except Exception as e:
                print(f"Failed to fetch roster for {team_name} for {season}: {str(e)}")

    return all_rosters

# Fetch rosters for all teams and seasons
all_nba_rosters = get_all_teams_rosters(all_teams, seasons)

# Filter to include only necessary columns for clarity
filtered_rosters = all_nba_rosters[['TeamName', 'Season', 'PLAYER', 'POSITION',  'HEIGHT','WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP']]

# Output or save the results
print(filtered_rosters.head())
filtered_rosters.to_csv('nba_team_rosters.csv', index=False)


Processing teams:   0%|          | 0/30 [00:00<?, ?it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonteamroster?LeagueID=&Season=2022-23&TeamID=1610612737 HTTP/1.1" 200 1684
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID', 'HOW_ACQUIRED']
Headers (simple): ['TEAM_ID', 'SEASON', 'COACH_ID', 'FIRST_NAME', 'LAST_NAME', 'COACH_NAME', 'IS_ASSISTANT', 'COACH_TYPE', 'SORT_SEQUENCE', 'SUB_SORT_SEQUENCE']


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonteamroster?LeagueID=&Season=2023-24&TeamID=1610612737 HTTP/1.1" 200 1764
Processing teams:   3%|▎         | 1/30 [00:07<03:43,  7.70s/it]

Headers (simple): ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID', 'HOW_ACQUIRED']
Headers (simple): ['TEAM_ID', 'SEASON', 'COACH_ID', 'FIRST_NAME', 'LAST_NAME', 'COACH_NAME', 'IS_ASSISTANT', 'COACH_TYPE', 'SORT_SEQUENCE', 'SUB_SORT_SEQUENCE']


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonteamroster?LeagueID=&Season=2022-23&TeamID=1610612738 HTTP/1.1" 200 1481
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


...

Headers (simple): ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID', 'HOW_ACQUIRED']
Headers (simple): ['TEAM_ID', 'SEASON', 'COACH_ID', 'FIRST_NAME', 'LAST_NAME', 'COACH_NAME', 'IS_ASSISTANT', 'COACH_TYPE', 'SORT_SEQUENCE', 'SUB_SORT_SEQUENCE']


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonteamroster?LeagueID=&Season=2022-23&TeamID=1610612766 HTTP/1.1" 200 1502
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID', 'HOW_ACQUIRED']
Headers (simple): ['TEAM_ID', 'SEASON', 'COACH_ID', 'FIRST_NAME', 'LAST_NAME', 'COACH_NAME', 'IS_ASSISTANT', 'COACH_TYPE', 'SORT_SEQUENCE', 'SUB_SORT_SEQUENCE']


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonteamroster?LeagueID=&Season=2023-24&TeamID=1610612766 HTTP/1.1" 200 1573
Processing teams: 100%|██████████| 30/30 [03:56<00:00,  7.88s/it]

Headers (simple): ['TeamID', 'SEASON', 'LeagueID', 'PLAYER', 'NICKNAME', 'PLAYER_SLUG', 'NUM', 'POSITION', 'HEIGHT', 'WEIGHT', 'BIRTH_DATE', 'AGE', 'EXP', 'SCHOOL', 'PLAYER_ID', 'HOW_ACQUIRED']
Headers (simple): ['TEAM_ID', 'SEASON', 'COACH_ID', 'FIRST_NAME', 'LAST_NAME', 'COACH_NAME', 'IS_ASSISTANT', 'COACH_TYPE', 'SORT_SEQUENCE', 'SUB_SORT_SEQUENCE']
        TeamName   Season            PLAYER POSITION HEIGHT WEIGHT  \
0  Atlanta Hawks  2022-23  Donovan Williams        G    6-6    190   
1  Atlanta Hawks  2022-23    Bruno Fernando      F-C   6-10    240   
2  Atlanta Hawks  2022-23     Jalen Johnson        F    6-8    219   
3  Atlanta Hawks  2022-23     Trent Forrest        G    6-4    210   
4  Atlanta Hawks  2022-23     Aaron Holiday        G    6-0    185   

     BIRTH_DATE   AGE EXP  
0  SEP 06, 2001  21.0   R  
1  AUG 15, 1998  24.0   3  
2  DEC 18, 2001  21.0   1  
3  JUN 12, 1998  25.0   2  
4  SEP 30, 1996  26.0   4  


In [3]:
from nba_api.stats.static import teams
from tqdm import tqdm
import pandas as pd

# Fetch all NBA teams
nba_teams = teams.get_teams()
seasons = ['2021-22', '2022-23']

def fetch_all_team_rosters(seasons):
    all_rosters = pd.DataFrame()
    for team in tqdm(nba_teams, desc='Teams Progress'):
        team_id = team['id']
        for season in tqdm(seasons, desc='Seasons Progress', leave=False):
            roster = get_roster(team_id, season)
            if roster is not None:
                roster['Season'] = season
                all_rosters = pd.concat([all_rosters, roster], ignore_index=True)
    return all_rosters

# Now, fetch and print the rosters using the improved function with retry and increased timeout
all_rosters = fetch_all_team_rosters(seasons)
print(all_rosters[['TEAM_ID', 'PLAYER', 'PLAYER_ID', 'POSITION', 'Season']])


Teams Progress:   0%|          | 0/30 [00:00<?, ?it/s]DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Timeout occurred for team 1610612737 season 2021-22, attempt 1


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Timeout occurred for team 1610612737 season 2021-22, attempt 2


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


...

Timeout occurred for team 1610612745 season 2022-23, attempt 3
All 3 attempts failed for team 1610612745 season 2022-23


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Timeout occurred for team 1610612746 season 2021-22, attempt 1


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Timeout occurred for team 1610612746 season 2021-22, attempt 2


In [11]:
print(shots_frame)

NameError: name 'shots_frame' is not defined